In [1]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import sys, re
import plotly
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from jitcache import Cache

#load data
survey_data = pd.read_csv(r'C:\Users\Vishaal Bhardwaj\Google File Stream\NLP Analysis\survey_data.csv')
def parent_desktop(row):
    platform = row['platform']
    
    if row['is_mobile'] == 0:
        if bool(re.search('Mac', str(platform))) == True:
            return 'Mac OS'
        elif bool(re.search('Linux', str(platform))) == True:
            return 'Linux'
        elif bool(re.search('Windows', str(platform))) == True:
            return 'Windows'
        else:
            return 'NaN'
survey_data['parent_desktop'] = survey_data.apply(lambda row: parent_desktop(row), axis=1)
survey_pv = pd.pivot_table(survey_data, index=['source'], columns=['parent_desktop'], values=['rating'], aggfunc=np.mean, fill_value=0)
#trace1 = go.Bar(x=survey_pv.index, y=survey_pv[('rating', 'Linux')], name='Linux')
#trace2 = go.Bar(x=survey_pv.index, y=survey_pv[('rating', 'Mac OS')], name='Mac OS')
#trace3 = go.Bar(x=survey_pv.index, y=survey_pv[('rating', 'NaN')], name='NaN')
#trace4 = go.Bar(x=survey_pv.index, y=survey_pv[('rating', 'Windows')], name='Windows')



#build app
app = JupyterDash(__name__)

app.layout = Div ([
    
    H3('Platform Rating'),
    
    Dropdown(
        id='my-dropdown',
        options = [
            {'label': 'linux', 'value': 'LINUX'},
            {'label': 'Mac OS', 'value': 'MAC OS'},
            {'label': 'NaN', 'value': 'NAN'},
            {'label': 'Windows', 'value': 'WINDOWS'}
        ]
    )
    
])


@app.react('my-graph', ['my-dropdown'])
def update_graph(dropdown_properties):
    selected_value = dropdown_properties['value']
    
    df = survey_pv
    
    return {
        'figure': go.Figure(
        data = [
            Scatter(
                x=df.rating,
                y=df.source,
                name = selected_value
            )
        ])
    }




app.run_server(mode='external')

In [2]:
survey_data = pd.read_csv('https://raw.githubusercontent.com/john-min/minsanity-medisprout/main/survey_data.csv?token=ADNMWRHNTTWDVHJKPLUQLSK7ZFOHQ')
survey_data['created_on'] = pd.to_datetime(survey_data['created_on'], format='%Y-%m-%d %H:%M:%S.%f')
survey_data['created_on_date'] = survey_data['created_on'].dt.date
survey_data['months'] = pd.to_datetime(survey_data['created_on'], format='%Y-%m-%d %H:%M:%S.%f').dt.strftime('%Y-%m')

In [3]:
mom_visits = survey_data.groupby('months')['visit_id'].count().to_frame().reset_index()
months = mom_visits['months'].tolist()
visits = mom_visits['visit_id'].tolist()

In [4]:
mom_rating = survey_data.groupby(['months', 'source', 'rating'])['visit_id'].count().reset_index()
mom_rating.rename(columns={'visit_id': 'visits'})
mom_rating['rating'] = mom_rating['rating'].astype(int)
mom_rating

,months,source,rating,visit_id
0,2020-07,Clinician,1,231
1,2020-07,Clinician,2,105
2,2020-07,Clinician,3,216
3,2020-07,Clinician,4,421
4,2020-07,Clinician,5,712
5,2020-07,Patient,1,47
6,2020-07,Patient,2,36
7,2020-07,Patient,3,110
8,2020-07,Patient,4,337
9,2020-07,Patient,5,1042


data = [go.Scatter(
    x=mom_visits.months,
    y=mom_visits['visit_id'],
    mode = 'lines+markers',
    name = 'lines+markers'
)]


plotly.offline.plot(data)

app = JupyterDash(__name__)
df = mom_rating

#---------------------------------------------------------------
app.layout = html.Div([

    html.Div([
        dcc.Graph(id='our_graph')
    ],className='nine columns'),

    html.Div([

        html.Br(),
        html.Div(id='output_data'),
        html.Br(),

        html.Label(['Choose column:'],style={'font-weight': 'bold', "text-align": "center"}),

        dcc.Dropdown(id='my_dropdown',
            options=[
                {'label': 'July 2020', 'value': '2020-07'},
                {'label': 'Aug 2020', 'value': '2020-08'},
                {'label': 'Sept 2020', 'value': '2020-09'},
                {'label': 'Oct 2020', 'value': '2020-10'},
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='July 2020',                    #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"100%"},             #use dictionary to define CSS styles of your dropdown
            # className='select_box',           #activate separate CSS document in assets folder
            # persistence=True,                 #remembers dropdown value. Used with persistence_type
            # persistence_type='memory'         #remembers dropdown value selected until...
            ),                                  #'memory': browser tab is refreshed
                                                #'session': browser tab is closed
                                                #'local': browser cookies are deleted
    ],className='three columns'),

])

#---------------------------------------------------------------
# Connecting the Dropdown values to the graph
@app.callback(
    Output(component_id='our_graph', component_property='figure'),
    [Input(component_id='my_dropdown', component_property='value')]
)


def build_graph(column_chosen):
    dff=df
    fig = px.pie(dff, names=column_chosen)
    fig.update_traces(textinfo='percent+label')
    fig.update_layout(title={'text': 'Visits per Rating Value',
                            'font': {'size': 28}, 'x': 0.5, 'xanchor': 'center'})

    return fig



#app.run_server(debug=True, port=8052)

#python IDE code
app = dash.Dash(__name__)

if name == '__main__':
    app.run_server(debug=True)

In [20]:
app = JupyterDash(__name__)
df = mom_rating
app.layout = html.Div([

    html.Div([
        dcc.Graph(id='our_graph')
    ],className='three columns'),

    html.Div([

        html.Br(),
        html.Label(['Rating'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='rating',
            options=[{'label':x, 'value':x} for x in mom_rating.sort_values('rating')['rating'].unique()],
            value='1',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose Value...',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'),
        
        html.Br(),
        html.Label(['Source'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='source',
            options=[{'label':x, 'value':x} for x in mom_rating.sort_values('source')['source'].unique()],
            value='Patient',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose Value...',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'),


    ],className='one column'),

])

#--------------------------------------------------------------- 

@app.callback(
    Output('our_graph','figure'),
    [Input('rating','value'),
    Input('source','value')], prevent_initial_call=True
)

def build_graph(rating_val, source_val):
    dff=mom_rating[(mom_rating['rating']==rating_val)|
                   (mom_rating['source']==source_val)]
    # print(dff[:5])

    fig = px.line(dff, x="months", y="visit_id",color='source', height=600)
    fig.update_layout(yaxis={'title':'Visits per rating'},
                      title={'text':'MoM Ratings',
                      'font':{'size':28},'x':0.5,'xanchor':'center'})
    return fig

#def update_graph(rating_val, source_val):
#    dff=mom_rating[(mom_rating['rating']==rating_val)|
#                   (df['rating']==source_val)]
#    if dff == 'Patient':
        

    
    
    
    
app.run_server(mode='external', debug=True, port=8055)

Dash app running on http://127.0.0.1:8055/


In [6]:
options=[{'label':x, 'value':x} for x in mom_rating.sort_values('rating')['rating'].unique()]
options

[{'label': 0, 'value': 0},
 {'label': 1, 'value': 1},
 {'label': 2, 'value': 2},
 {'label': 3, 'value': 3},
 {'label': 4, 'value': 4},
 {'label': 5, 'value': 5}]

In [7]:
def build_graph(first_rating):
    dff=mom_rating[(mom_rating['rating']==first_rating)]
    print(dff)

In [8]:
build_graph(3.0)

     months     source  rating  visit_id
2   2020-07  Clinician       3       216
7   2020-07    Patient       3       110
13  2020-08  Clinician       3       275
18  2020-08    Patient       3       419
24  2020-09  Clinician       3       384
29  2020-09    Patient       3       934
34  2020-10  Clinician       3        77
39  2020-10    Patient       3       185


In [15]:
dff=mom_rating[(mom_rating['rating']==1)|
                   (mom_rating['rating']=='Patient')]

dff

,months,source,rating,visit_id
0,2020-07,Clinician,1,231
5,2020-07,Patient,1,47
11,2020-08,Clinician,1,263
16,2020-08,Patient,1,323
22,2020-09,Clinician,1,394
27,2020-09,Patient,1,637
32,2020-10,Clinician,1,79
37,2020-10,Patient,1,116
